# Single-Agent Team Example

This notebook demonstrates how to run a single agent in a team configuration using AutoGen. This approach allows the agent to perform multiple iterations of tool calls until a termination condition is met.

## Example Scenario

The agent will increment a number from a starting value until it reaches a target value, using a tool to perform the increment.


In [1]:
# Install dependencies
!pip install --quiet -U "autogen-agentchat>=0.7" "autogen-ext[openai,mcp]>=0.7" "mcp>=1.0.0"

# IMPORTANT: See: https://github.com/microsoft/autogen/issues/6906
!pip install --quiet --force-reinstall "openai==1.80"

> **⚠️ IMPORTANT:**  
> If you just ran the `!pip install --quiet --force-reinstall "openai==1.80"` command,  
> you **must** restart the Jupyter kernel before continuing.  
> This ensures the newly installed `openai` package is loaded into memory  
> and avoids mixed-version issues that can cause runtime errors.  
>  
> **In Jupyter:** go to **Kernel → Restart & Clear Output**, then rerun the notebook from the top.

In [2]:
import asyncio
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import TextMessageTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_ext.models.openai import OpenAIChatCompletionClient

# Initialize the model client
model_client = OpenAIChatCompletionClient(
    model="gpt-4o",
    parallel_tool_calls=False,  # Disable parallel tool calls for this example
)

# Define a simple increment tool
def increment_tool(number: int) -> int:
    """Increment the input number by 1."""
    return number + 1

# Create the single agent with the tool
single_agent = AssistantAgent(
    "incrementer",
    model_client=model_client,
    tools=[increment_tool],
    system_message="You are an assistant that increments numbers using the provided tool.",
)

# Define termination condition
termination_condition = TextMessageTermination("incrementer")

# Create a single-agent team
team = RoundRobinGroupChat(
    [single_agent],
    termination_condition=termination_condition,
)

# Example usage (to be run in an async context):
async for message in team.run_stream(task="Increment the number 3 to 7."):
    print(message)

await model_client.close()

id='1f0c1da9-e1a5-43eb-9cf1-3da1c1399f56' source='user' models_usage=None metadata={} created_at=datetime.datetime(2025, 8, 9, 6, 30, 34, 181011, tzinfo=datetime.timezone.utc) content='Increment the number 3 to 7.' type='TextMessage'
id='2fed573c-8d32-4105-a1ae-c5d4755f6136' source='incrementer' models_usage=RequestUsage(prompt_tokens=73, completion_tokens=14) metadata={} created_at=datetime.datetime(2025, 8, 9, 6, 30, 34, 824605, tzinfo=datetime.timezone.utc) content=[FunctionCall(id='call_S4qVnfv6CAVzX3nLzImQL0s8', arguments='{"number":3}', name='increment_tool')] type='ToolCallRequestEvent'
id='fb660b6d-2834-411e-bf3b-fb44343c0897' source='incrementer' models_usage=None metadata={} created_at=datetime.datetime(2025, 8, 9, 6, 30, 34, 829450, tzinfo=datetime.timezone.utc) content=[FunctionExecutionResult(content='4', name='increment_tool', call_id='call_S4qVnfv6CAVzX3nLzImQL0s8', is_error=False)] type='ToolCallExecutionEvent'
id='09a78a99-59b7-465a-bf1c-097c80f37032' source='increment